In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import scanpy as sc
import scipy.sparse as sp
import SpatialEx.preprocess as pp
from SpatialEx.utils import Compute_metrics, Compute_MoransI
from SpatialEx.SpatialEx_pyG import Train_SpatialEx, Train_SpatialExP, Train_SpatialExP_Big

# **1. SpatialEx Translates Histology to Omics at Single-Cell Resolution**

## 1.1 Train the model

In [ ]:
# preprocess slice 1
file_path1 = './datasets/Human_Breast_Cancer_Rep1/cell_feature_matrix.h5'
obs_path1 = './datasets/Human_Breast_Cancer_Rep1/cells.csv'
img_path1 = './datasets/Human_Breast_Cancer_Rep1/Xenium_FFPE_Human_Breast_Cancer_Rep1_he_image.ome.tif'
transform_mtx_path1 = './datasets/Human_Breast_Cancer_Rep1/Xenium_FFPE_Human_Breast_Cancer_Rep1_he_imagealignment.csv'
resolution = 64
device = 'cuda'

adata1 = pp.Read_Xenium(file_path1, obs_path1)
adata1 = pp.Preprocess_adata(adata1)
img, scale = pp.Read_HE_image(img_path1)
print('read H&E image successfully')

transform_mtx = pd.read_csv(transform_mtx_path1, header=None).values
adata1 = pp.Register_physical_to_pixel(adata1, transform_mtx, scale=scale)
he_patches, adata1 = pp.Tiling_HE_patches(resolution, adata1, img)

# put the pretrained weights of the image encoder
image_encoder_dir = './image_encoder/' 
os.makedirs(image_encoder_dir, exist_ok=True)  # create directory if it does not exist

# When using the uni encoder, make sure to place the 'pytorch_model.bin' file in the 'image_encoder' folder. 
# Here, we implement the uni, gigapath, phikon, resnet50, resnet101, and resnet152.
# To use a different image encoder, simply edit the 'pp.Extract_HE_patches_representaion' function.
adata1 = pp.Extract_HE_patches_representaion(he_patches, store_key='he', adata=adata1, image_encoder='uni', \
                                            device=device)

# To save time, we provide preprocessed H&E visual features, available in Tutorial 1. 
# Just replace the code above with this one-liner.
# adata1 = sc.read_h5ad(preprocessed_slice_one)

# preprocess slice 2
file_path2 = './datasets/Human_Breast_Cancer_Rep2/cell_feature_matrix.h5'
obs_path2 = './datasets/Human_Breast_Cancer_Rep2/cells.csv'
img_path2 = './datasets/Human_Breast_Cancer_Rep2/Xenium_FFPE_Human_Breast_Cancer_Rep2_he_image.ome.tif'
transform_mtx_path2 = './datasets/Human_Breast_Cancer_Rep2/Xenium_FFPE_Human_Breast_Cancer_Rep2_he_imagealignment.csv'

adata2 = pp.Read_Xenium(file_path2, obs_path2)    
adata2 = pp.Preprocess_adata(adata2)
img, scale = pp.Read_HE_image(img_path2)
print('read H&E image successfully')

transform_mtx = pd.read_csv(transform_mtx_path2, header=None).values
adata2 = pp.Register_physical_to_pixel(adata2, transform_mtx, scale=scale)
he_patches, adata2 = pp.Tiling_HE_patches(resolution, adata2, img)
adata2 = pp.Extract_HE_patches_representaion(he_patches, store_key='he', adata=adata2, image_encoder='uni', \
                                            device=device)
# Similarly, if you would like to use our preprocessed visual features, you can apply the following code snippet.
# adata2 = sc.read_h5ad(preprocess_slice_two)

# use the SpatialEx
model = Train_SpatialEx(adata1, adata2, device=device, epochs=500)
model.train()

## 1.2 Compute the metrics

In [ ]:
panel_1b = pd.read_csv('./results/HE_to_omics_panel1b.csv', index_col=0)
panel_2a = pd.read_csv('./results/HE_to_omics_panel2a.csv', index_col=0)
panel_1b.index, panel_2a.index = panel_1b.index.astype(str), panel_2a.index.astype(str)

# compute the metrics for slice 1
graph = pp.Build_graph(adata1.obsm['spatial'], graph_type='knn', weighted='gaussian', \
                       apply_normalize='row', type='coo')
cs_sg, cs_reduce_sg = Compute_metrics(adata1.X, panel_1b, metric='cosine_similarity')
ssim, ssim_reduce = Compute_metrics(adata1.X, panel_1b, metric='ssim', graph=graph)
pcc, pcc_reduce = Compute_metrics(adata1.X, panel_1b, metric='pcc')
cmd, cmd_reduce = Compute_metrics(adata1.X, panel_1b, metric='cmd')
print('Evaluation of the Slice1 in gene-level, cosine similarity: ', cs_reduce_sg, \
      ' ssim: ', ssim_reduce, ' pcc: ', pcc_reduce, ' cmd: ', cmd_reduce)     

# compute the metrics for slice 2
graph = pp.Build_graph(adata2.obsm['spatial'], graph_type='knn', weighted='gaussian', \
                       apply_normalize='row', type='coo')
cs_sg, cs_reduce_sg = Compute_metrics(adata2.X, panel_2a, metric='cosine_similarity')
ssim, ssim_reduce = Compute_metrics(adata2.X, panel_2a, metric='ssim', graph=graph)
pcc, pcc_reduce = Compute_metrics(adata2.X, panel_2a, metric='pcc')
cmd, cmd_reduce = Compute_metrics(adata2.X, panel_2a, metric='cmd')
print('Evaluation of the Slice2 in gene-level, cosine similarity: ', cs_reduce_sg, \
      ' ssim: ', ssim_reduce, ' pcc: ', pcc_reduce, ' cmd: ', cmd_reduce)

## **2. SpatialEx+ Enables Larger Panel Spatial Analysis through Panel Diagonal Integration**

### 2.1 Train the model

In [ ]:
# preprocess slice 1
file_path1 = './datasets/Human_Breast_Cancer_Rep1/cell_feature_matrix.h5'
obs_path1 = './datasets/Human_Breast_Cancer_Rep1/cells.csv'
img_path1 = './datasets/Human_Breast_Cancer_Rep1/Xenium_FFPE_Human_Breast_Cancer_Rep1_he_image.ome.tif'
transform_mtx_path1 = './datasets/Human_Breast_Cancer_Rep1/Xenium_FFPE_Human_Breast_Cancer_Rep1_he_imagealignment.csv'
resolution = 64
device = 'cuda'

# set the panel
selection = pd.read_csv('./datasets/Selection_by_name.csv', index_col=0)
panelA = selection.index[selection['slice1']].tolist()
panelB = selection.index[selection['slice2']].tolist()

adata1 = pp.Read_Xenium(file_path1, obs_path1)
adata1 = pp.Preprocess_adata(adata1, selected_genes=panelA)
img, scale = pp.Read_HE_image(img_path1)

transform_mtx = pd.read_csv(transform_mtx_path1, header=None).values
adata1 = pp.Register_physical_to_pixel(adata1, transform_mtx, scale=scale)
he_patches, adata1 = pp.Tiling_HE_patches(resolution, adata1, img)
adata1 = pp.Extract_HE_patches_representaion(he_patches, store_key='he', adata=adata1, image_encoder='uni', \
                                            device=device)
# If you would like to use our preprocessed visual features, you can apply the following code snippet.
# adata1 = sc.read_h5ad(preprocess_slice_one)

# preprocess slice 2
file_path2 = './datasets/Human_Breast_Cancer_Rep2/cell_feature_matrix.h5'
obs_path2 = './datasets/Human_Breast_Cancer_Rep2/cells.csv'
img_path2 = './datasets/Human_Breast_Cancer_Rep2/Xenium_FFPE_Human_Breast_Cancer_Rep2_he_image.ome.tif'
transform_mtx_path2 = './datasets/Human_Breast_Cancer_Rep2/Xenium_FFPE_Human_Breast_Cancer_Rep2_he_imagealignment.csv'

adata2 = pp.Read_Xenium(file_path2, obs_path2)
adata2 = pp.Preprocess_adata(adata2, selected_genes=panelB)
img, scale = pp.Read_HE_image(img_path2)

transform_mtx = pd.read_csv(transform_mtx_path2, header=None).values
adata2 = pp.Register_physical_to_pixel(adata2, transform_mtx, scale=scale)
he_patches, adata2 = pp.Tiling_HE_patches(resolution, adata2, img)
adata2 = pp.Extract_HE_patches_representaion(he_patches, store_key='he', adata=adata2, image_encoder='uni', \
                                            device=device)
# Similarly, if you would like to use our preprocessed visual features, you can apply the following code snippet.
# adata2 = sc.read_h5ad(preprocess_slice_two)

# use the SpatialEx+
model = Train_SpatialExP(adata1, adata2, device=device, epochs=500)
model.train()

### 2.2 Compute the metrics

In [ ]:
panel_1b = np.load('./results/omics1.npy')
panel_2a = np.load('./results/omics2.npy')
index_1 = adata1.obs_names
index_2 = adata2.obs_names

# build the ground truth for panelB of slice 1
adata1 = pp.Read_Xenium(file_path1, obs_path1)
adata1 = pp.Preprocess_adata(adata1, cell_mRNA_cutoff=0, selected_genes=panelB)
adata1 = adata1[index_1]

# compute the metrics for slice 1
graph = pp.Build_graph(adata1.obsm['spatial'], graph_type='knn', weighted='gaussian', \
                       apply_normalize='row', type='coo')
cs_sg, cs_reduce_sg = Compute_metrics(adata1.X, panel_1b, metric='cosine_similarity')
ssim, ssim_reduce = Compute_metrics(adata1.X, panel_1b, metric='ssim', graph=graph)
pcc, pcc_reduce = Compute_metrics(adata1.X, panel_1b, metric='pcc')
cmd, cmd_reduce = Compute_metrics(adata1.X, panel_1b, metric='cmd')
print('Evaluation of the Slice1 in gene-level, cosine similarity: ', cs_reduce_sg, \
      ' ssim: ', ssim_reduce, ' pcc: ', pcc_reduce, ' cmd: ', cmd_reduce)     

# build the ground truth for panelA of slice 2
adata2 = pp.Read_Xenium(file_path2, obs_path2)
adata2 = pp.Preprocess_adata(adata2, cell_mRNA_cutoff=0, selected_genes=panelA)
adata2 = adata2[index_2]

# compute the metrics for slice 2
graph = pp.Build_graph(adata2.obsm['spatial'], graph_type='knn', weighted='gaussian', \
                       apply_normalize='row', type='coo')
cs_sg, cs_reduce_sg = Compute_metrics(adata2.X, panel_2a, metric='cosine_similarity')
ssim, ssim_reduce = Compute_metrics(adata2.X, panel_2a, metric='ssim', graph=graph)
pcc, pcc_reduce = Compute_metrics(adata2.X, panel_2a, metric='pcc')
cmd, cmd_reduce = Compute_metrics(adata2.X, panel_2a, metric='cmd')
print('Evaluation of the Slice2 in gene-level, cosine similarity: ', cs_reduce_sg, \
      ' ssim: ', ssim_reduce, ' pcc: ', pcc_reduce, ' cmd: ', cmd_reduce)

## **3. Scalability on Million-Cell Tissue Sections**

### 3.1 Train the model

In [ ]:
# set the panel
panel_selection = pd.read_csv('./datasets/Big_by_name.csv', index_col=0)
panelA = panel_selection.index[panel_selection['panelA']==1]
panelB = panel_selection.index[panel_selection['panelB']==1]

# preprocess slice 1
file_path1 = './datasets/Human_Breast_IDC_Big1/cell_feature_matrix.h5'
obs_path1 = './datasets/Human_Breast_IDC_Big1/cells.csv'
img_path1 = './datasets/Human_Breast_IDC_Big1/Xenium_V1_FFPE_Human_Breast_IDC_Big_1_he_unaligned_image.ome.tif'
transform_mtx_path1 = './datasets/Human_Breast_IDC_Big1/Xenium_V1_FFPE_Human_Breast_IDC_Big_1_he_unaligned_image_matrix.csv'
resolution = 64
device = 'cuda'

adata1 = pp.Read_Xenium(file_path1, obs_path1)
adata1 = pp.Preprocess_adata(adata1, selected_genes=panelA)
panel_A1 = torch.Tensor(adata1.X)

img, scale = pp.Read_HE_image(img_path1)
trans_mtx = pd.read_csv(transform_mtx_path1, header=None).values
adata1 = pp.Register_physical_to_pixel(adata1, trans_mtx, scale)
patches, adata1 = pp.Tiling_HE_patches(resolution, adata1, img)
adata1 = pp.Extract_HE_patches_representaion(patches, store_key='he', adata=adata1, image_encoder='uni', \
                                            device=device)

# preprocess slice 2
file_path2 = './datasets/Human_Breast_IDC_Big2/cell_feature_matrix.h5'
obs_path2 = './datasets/Human_Breast_IDC_Big2/cells.csv'
img_path2 = './datasets/Human_Breast_IDC_Big2/Xenium_V1_FFPE_Human_Breast_IDC_Big_2_he_unaligned_image.ome.tif'
transform_mtx_path2 = './datasets/Human_Breast_IDC_Big2/Xenium_V1_FFPE_Human_Breast_IDC_Big_2_he_imagealignment.csv'

adata2 = pp.Read_Xenium(file_path2, obs_path2)
adata2 = pp.Preprocess_adata(adata2, selected_genes=panelB)
panel_B2 = torch.Tensor(adata2.X)

img, scale = pp.Read_HE_image(img_path2)
trans_mtx = pd.read_csv(transform_mtx_path2, header=None).values
adata2 = pp.Register_physical_to_pixel(adata2, trans_mtx, scale)
patches, adata1 = pp.Tiling_HE_patches(resolution, adata2, img)
adata2 = pp.Extract_HE_patches_representaion(patches, store_key='he', adata=adata2, image_encoder='uni', \
                                            device=device)

# use the SpatialEx+
model = Train_SpatialExP_Big(adata1, adata2, device=device, epochs=500, batch_num=50)
model.train()

### 3.2 Compute the metrics

In [ ]:
panelB1 = np.load('./results/big_panel_B1.npy')
panelA2 = np.load('./results/big_panel_A2.npy')
index_1 = adata1.obs_names
index_2 = adata2.obs_names

# build the ground truth for panelB of slice 1
adata1 = pp.Read_Xenium(file_path1, obs_path1)
adata1 = pp.Preprocess_adata(adata1, cell_mRNA_cutoff=0, selected_genes=panelB)
adata1 = adata1[index_1]

# compute the metrics for slice 1
graph = pp.Build_graph(adata1.obsm['spatial'], graph_type='knn', weighted='gaussian', \
                       apply_normalize='row', type='csr')
cs_sg, cs_reduce_sg = Compute_metrics(adata1.X, panelB1, metric='cosine_similarity')
ssim, ssim_reduce = Compute_metrics(adata1.X, panelB1, metric='ssim', graph=graph)
pcc, pcc_reduce = Compute_metrics(adata1.X, panelB1, metric='pcc')
cmd, cmd_reduce = Compute_metrics(adata1.X, panelB1, metric='cmd')
print('Evaluation of the Slice1 in gene-level, cosine similarity: ', cs_reduce_sg, \
      ' ssim: ', ssim_reduce, ' pcc: ', pcc_reduce, ' cmd: ', cmd_reduce)     

# build the ground truth for panelA of slice 2
adata2 = pp.Read_Xenium(file_path2, obs_path2)
adata2 = pp.Preprocess_adata(adata2, cell_mRNA_cutoff=0, selected_genes=panelA)
adata2 = adata1[index_2]

# compute the metrics for slice 2
graph = pp.Build_graph(adata2.obsm['spatial'], graph_type='knn', weighted='gaussian', \
                       apply_normalize='row', type='csr')
cs_sg, cs_reduce_sg = Compute_metrics(adata2.X, panelA2, metric='cosine_similarity')
ssim, ssim_reduce = Compute_metrics(adata2.X, panelA2, metric='ssim', graph=graph)
pcc, pcc_reduce = Compute_metrics(adata2.X, panelA2, metric='pcc')
cmd, cmd_reduce = Compute_metrics(adata2.X, panelA2, metric='cmd')
print('Evaluation of the Slice2 in gene-level, cosine similarity: ', cs_reduce_sg, \
      ' ssim: ', ssim_reduce, ' pcc: ', pcc_reduce, ' cmd: ', cmd_reduce)

## **SpatialEx+ Enables Spatial Multi-omics through Omics Diagonal Integration**

### **4. Transcriptomics-Proteomics**

#### 4.1 Train the model

In [ ]:
# preprocess the transcriptomics
file_path1 = './datasets/Human_Breast_Cancer_Rep2/cell_feature_matrix.h5'
obs_path1 = './datasets/Human_Breast_Cancer_Rep2/cells.csv'
img_path1 = './datasets/Human_Breast_Cancer_Rep2/Xenium_FFPE_Human_Breast_Cancer_Rep2_he_image.ome.tif'
transform_mtx_path1 = './datasets/Human_Breast_Cancer_Rep2/Xenium_FFPE_Human_Breast_Cancer_Rep2_he_imagealignment.csv'
resolution = 64
device = 'cuda'

adata1 = pp.Read_Xenium(file_path1, obs_path1)
adata1 = pp.Preprocess_adata(adata1)
img, scale = pp.Read_HE_image(img_path1)
transform_mtx = pd.read_csv(transform_mtx_path1, header=None).values
adata1 = pp.Register_physical_to_pixel(adata1, transform_mtx, scale)
he_patches, adata1 = pp.Tiling_HE_patches(resolution, adata1, img)
adata1 = pp.Extract_HE_patches_representaion(he_patches, 'he', adata1, image_encoder='uni', \
                                            device=device)

# preprocess the proteomics
file_path2 = './datasets/Human_Breast_Cancer_Rep1/cell_protein_matrix.h5ad'
obs_path2 = './datasets/Human_Breast_Cancer_Rep1/cells.csv'
img_path2 = './datasets/Human_Breast_Cancer_Rep1/Xenium_FFPE_Human_Breast_Cancer_Rep1_he_image.ome.tif'
transform_mtx_path2 = './datasets/Human_Breast_Cancer_Rep1/Xenium_FFPE_Human_Breast_Cancer_Rep1_he_imagealignment.csv'

adata2 = sc.read_h5ad(file_path2)
adata2.var_names = adata2.var_names.astype(str)
adata2.obs_names = adata2.obs_names.astype(str)
obs = pd.read_csv(obs_path2, index_col=0)
obs.index = obs.index.astype(str)
adata2 = adata2[obs.index]
adata2.obs = obs
adata2.obsm['spatial'] = adata2.obs[['x_centroid', 'y_centroid']].values
adata2.var_names_make_unique()
sc.pp.scale(adata2)

img, scale = pp.Read_HE_image(img_path2)
trans_mtx = pd.read_csv(transform_mtx_path2, header=None).values
adata2 = pp.Register_physical_to_pixel(adata2, trans_mtx, scale)
he_patches, adata2 = pp.Tiling_HE_patches(resolution, adata2, img)
adata2 = pp.Extract_HE_patches_representaion(he_patches, 'he', adata2, image_encoder='uni', \
                                            device=device)   

# use the SpatialEx+
model = Train_SpatialExP(adata1, adata2, device=device, epochs=500)
model.train()

#### 4.2 Compute the metrics

In [ ]:
panel_1b = np.load('./results/omics1.npy')
panel_2a = np.load('./results/omics2.npy')
index_2 = adata2.obs_names

# We can only compute the metrics for slice 2 due to the missing proteomics of slice 1
# build the ground truth for panelA of slice 2
file_path2 = './datasets/Human_Breast_Cancer_Rep1/cell_feature_matrix.h5'
obs_path2 = './datasets/Human_Breast_Cancer_Rep1/cells.csv'

adata2 = pp.Read_Xenium(file_path2, obs_path2)
adata2 = pp.Preprocess_adata(adata2, cell_mRNA_cutoff=0)
adata2 = adata2[index_2]

# compute the metrics for slice 2
graph = pp.Build_graph(adata2.obsm['spatial'], graph_type='knn', weighted='gaussian', \
                       apply_normalize='row', type='coo')
cs_sg, cs_reduce_sg = Compute_metrics(adata2.X, panel_2a, metric='cosine_similarity')
ssim, ssim_reduce = Compute_metrics(adata2.X, panel_2a, metric='ssim', graph=graph)
pcc, pcc_reduce = Compute_metrics(adata2.X, panel_2a, metric='pcc')
cmd, cmd_reduce = Compute_metrics(adata2.X, panel_2a, metric='cmd')
print('Evaluation of the transcriptomics of the Slice2 in gene-level, cosine similarity: ', cs_reduce_sg, \
      ' ssim: ', ssim_reduce, ' pcc: ', pcc_reduce, ' cmd: ', cmd_reduce)

### **5. Transcriptomics-Metabolomics**

#### 5.1 Train the model

In [ ]:
# preprocess the metabolomics
file_path1 = './datasets/Multi_modality/metabolite_V11L12-109_C1.h5ad'
img_path1 = './datasets/Multi_modality/220506_MSi_V11L12-109_C1.jpg'
resolution = 580
device = 'cuda'
num_features = [100, 2000] # The number of chosen genes for metabolomics and transcriptomics

adata1 = sc.read_h5ad(file_path1)
adata1.var_names = np.array(adata1.var['metabolism'].values)
adata1.var_names_make_unique()
adata1.obs_names_make_unique()
sc.pp.filter_genes(adata1, min_cells=1)
adata1 = pp.Preprocess_adata(adata1, cell_mRNA_cutoff=0, scale=True)
mz_selected = np.load('./datasets/Metabolites_panel.npy', allow_pickle=True)
adata1 = adata1[:, mz_selected]
spot_A1 = torch.Tensor(adata1.X)

img, _ = pp.Read_HE_image(img_path1, suffix='.jpg')
he_patches, adata1 = pp.Tiling_HE_patches(resolution, adata1, img, key='spatial')
adata1 = pp.Extract_HE_patches_representaion(he_patches, 'he', adata1, image_encoder='uni', \
                                            device=device)

# preprocess the transcriptomics
file_path2 = './datasets/Multi_modality/rna_V11L12-109_B1.h5ad'
img_path2 = './datasets/Multi_modality/220506_MSi_V11L12-109_B1.jpg'

adata2 = sc.read_h5ad(file_path2)
adata2.var_names_make_unique()
adata2.obs_names_make_unique()
sc.pp.filter_genes(adata2, min_cells=1)
sc.pp.log1p(adata2)
adata2 = pp.Preprocess_adata(adata2, cell_mRNA_cutoff=0, scale=True)
rna = np.load('./datasets/RNA_panel.npy', allow_pickle=True)
adata2 = adata2[:, rna]

img, _ = pp.Read_HE_image(img_path2, suffix='.jpg')
he_patches, adata2 = pp.Tiling_HE_patches(resolution, adata2, img, key='spatial')
adata2 = pp.Extract_HE_patches_representaion(he_patches, 'he', adata2, image_encoder='uni', \
                                            device=device)

# use the SpatialEx+
model = Train_SpatialExP(adata1, adata2, device=device, epochs=600)
model.train()

### 5.2 Compute the metrics

#### 5.2.1 Compute the metrics for Slice 1

In [ ]:
panelB1 = np.load('./results/omics1.npy')
panelB1 = np.maximum(panelB1, 0)

adata = sc.read_h5ad('./datasets/Multi_modality/rna_V11L12-109_C1.h5ad')
adata.var_names_make_unique()
adata.obs_names_make_unique()
sc.pp.filter_genes(adata, min_cells=1)
sc.pp.log1p(adata)
adata = pp.Preprocess_adata(adata, cell_mRNA_cutoff=0, scale=True)
adata = adata[adata1.obs_names, rna]

cs_sg, cs_reduce_sg = Compute_metrics(adata.X, panelB1, metric='cosine_similarity', reduce='mean')  
pcc, pcc_reduce = Compute_metrics(adata.X, panelB1, metric='pcc', reduce='mean')
cmd, cmd_reduce = Compute_metrics(adata.X, panelB1, metric='cmd', reduce='mean')
print('cosine similarity: ', cs_reduce_sg, ' pcc: ', pcc_reduce, 'cmd: ', cmd_reduce)

#### 5.2.2 Compute the metrics for Slice 2

In [ ]:
panelA2 = np.load('./results/omics2.npy')
panelA2 = np.maximum(panelA2, 0)

adata = sc.read_h5ad('./datasets/Multi_modality/metabolite_V11L12-109_B1.h5ad')
adata.var_names = np.array(adata.var['metabolism'].values)
adata.var_names_make_unique()
adata.obs_names_make_unique()
sc.pp.filter_genes(adata, min_cells=1)
adata = pp.Preprocess_adata(adata, cell_mRNA_cutoff=0, scale=True)
adata = adata[adata2.obs_names, mz_selected]

cs_sg, cs_reduce_sg = Compute_metrics(adata.X, panelA2, metric='cosine_similarity', reduce='mean')  
pcc, pcc_reduce = Compute_metrics(adata.X, panelA2, metric='pcc', reduce='mean')
cmd, cmd_reduce = Compute_metrics(adata.X, panelA2, metric='cmd', reduce='mean')
print('cosine similarity: ', cs_reduce_sg, ' pcc: ', pcc_reduce, 'cmd: ', cmd_reduce)